# Octopus Agile in 2020

Some data insights into Agile this past year.

In [1]:
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import Markdown

def read_year(i):
    df = pd.read_csv(f'agile.{i}.csv', index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert('Europe/London')
    return df

df = read_year(2020)
all_years = pd.concat(
    read_year(i) for i in (2017, 2018, 2019, 2020))

## Best and worst months

As lockdown commenced, electricity rates plunged to record lows.

Looking at the median rate per kWh by month:

In [2]:
median_rate = df['rate'].resample('MS').median()
cheapest = median_rate.idxmin()
expensive = median_rate.idxmax()
display(Markdown(f'The cheapest month was **{cheapest.strftime("%B")}** averaging **{median_rate.min()} p/kWh**.'))
display(Markdown(f'The most expensive month was **{expensive.strftime("%B")}** averaging **{median_rate.max()} p/kWh**.\n'))
alt.Chart(median_rate.reset_index()).mark_bar(width=15).encode(
    alt.X('timestamp', title='Month'),
    alt.Y('rate', title='Rate / p'),
    alt.Color('rate', scale=alt.Scale(scheme='goldred'), legend=None),
).properties(width=270)

The cheapest month was **May** averaging **5.733 p/kWh**.

The most expensive month was **December** averaging **11.466 p/kWh**.


alt.Chart(...)

In comparison to previous years, it was a very cheap year for Agile...

In [3]:
median_rate = all_years.resample('MS').median()
median_rate['year'] = median_rate.index.year
alt.Chart(median_rate.reset_index()).mark_bar(width=10).encode(
    alt.X('timestamp', title='Month'),
    alt.Y('rate', title='Rate / p'),
    alt.Color('rate', scale=alt.Scale(scheme='goldred'), legend=None),
    alt.Facet('year', columns=1, spacing=0),
).resolve_scale(
    x='independent'
).properties(
    width=220
)

alt.Chart(...)

With a record number of plunge days in 2020...

In [4]:
plunges = (all_years.rate < 0).resample('D').sum()
by_year = (plunges > 0).groupby(plunges.index.year).sum().astype(int)
alt.Chart(by_year.reset_index()).mark_bar().encode(
    alt.X('rate', title='Plunge days'),
    alt.Y('timestamp:O', title='Year'),
    color=alt.value('#008888'),
).properties(
    title='Plunge days by year',
    width=250,
    height=100,
)

alt.Chart(...)

In [5]:
plunges_2020 = (df.rate < 0).resample('D').sum() > 0
by_month = plunges_2020.resample('MS').sum()
alt.Chart(by_month.reset_index()).mark_bar(height=15).encode(
    alt.X('rate', title='Plunge days'),
    alt.Y('timestamp', title='Month'),
    alt.Color('timestamp', scale=alt.Scale(scheme='inferno'), legend=None),
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
).properties(
    title='Plunge days by month 2020',
    width=250,
    height=250,
)

alt.Chart(...)

With Sunday having the most plunges, closely followed by Monday. Thursday had none!

In [6]:
by_dow = plunges_2020.groupby(plunges_2020.index.day_name()).sum()
by_dow = by_dow.sort_values(ascending=False)
alt.Chart(by_dow.reset_index()).mark_bar().encode(
    alt.X('timestamp', title=None, sort=None),
    alt.Y('rate', title='Plunge days'),
    alt.Color('timestamp', legend=None),
).properties(
    title='Plunges by day of week',
    width=250,
    height=250,
)

alt.Chart(...)

Agile was cheaper than Go about 3/4 of the time.

In [7]:
hm = df.index.hour * 2 + df.index.minute/30
go_rates = np.where((hm >= 1) & (hm < 9), 4.9980, 14.1225) # 00:30 to 04:30
go_rates = pd.Series(go_rates, index=df.index)

comp = pd.DataFrame([
        ('Agile cheaper', (df.rate < go_rates).sum()),
        ('Go cheaper', (df.rate > go_rates).sum()),
    ], columns=['label', 'count']
)
alt.Chart(comp).mark_bar().encode(
    alt.X('count', title='Half hours'),
    alt.Y('label', title=None),
    color=alt.value("#FFAA00"),
).properties(
    width=250,
    height=80,
)



alt.Chart(...)

In [8]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>